In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, RectBivariateSpline

# Astronomy
import astropy
from astropy.units import au, minute

from typing import List, Optional

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_pos
from asteroid_spline import make_spline_ast_elt, make_spline_ast_pos, make_spline_df
from asteroid_direction import calc_dir_ast2obs
from planets_interp import get_earth_pos
from asteroid_direction import calc_dir_linear, calc_dir_spline
from asteroid_direction_test import jpl_ast_dir_populate
from ra_dec import radec2dir
from db_utils import sp2df, df2db


In [53]:
from utils import arange_inc

n0 = 0
n1 = 10
mjd0 = 48000
mjd1 = 63000
interval = 4

mpd = 1440.0
dpm = 1.0 / 1440


In [54]:
ast = sp2df(sp_name='KS.GetAsteroids')

In [80]:
# Start, end and step for TimeID
TimeID_0 = mjd0 * mpd
TimeID_1 = mjd1 * mpd
TimeID_interval = interval * mpd
# Array of TimeID
TimeIDs_one = arange_inc(TimeID_0, TimeID_1, TimeID_interval).astype(np.int64)

In [81]:
# Get array of asteroid IDs between n0 and n1
mask = (n0 <= ast.AsteroidID) & (ast.AsteroidID < n1)
asteroid_id_one = ast.AsteroidID[mask].values

In [82]:
# The number of observation times and asteroids
N_t = TimeIDs_one.shape[0]
N_ast = asteroid_id_one.shape[0]


In [83]:
TimeIDs_one

array([69120000, 69125760, 69131520, ..., 90708480, 90714240, 90720000])

In [88]:
# Tile TimeIDs
TimeIDs = np.tile(TimeIDs_one, N_ast)

In [89]:
# Array of observation times
t_obs = TimeIDs * dpm

In [92]:
np.repeat(asteroid_id_one, N_t)

array([1, 1, 1, ..., 9, 9, 9])

In [90]:
# Using geocentric observer so there is no topos adjustment
q_obs_one = get_earth_pos(ts=t_obs)


In [91]:
q_obs.shape

(7502, 3)

In [71]:
q_obs_one.shape


(3751, 3)

In [ ]:
    # Delegate to calc_dir_ast2obs()
    df = calc_dir_ast2obs(t_obs=t_obs, asteroid_id=asteroid_id, q_obs=q_obs, iters=2)

# Test asteroid direction with iterated splines

In [3]:
# Import the JPL asteroid directions with the JPL state vectors
df_jpl = sp2df(sp_name='JPL.AsteroidDirectionTest')
# df_jpl = sp2df(sp_name='JPL.AsteroidDirectionVectorTest')

In [4]:
# Mask to only geocenter
mask = (df_jpl.ObservatoryID==0)
df_jpl = df_jpl[mask].reset_index(drop=True)

In [5]:
# The Horizons DataFrame
df_jpl

,AsteroidID,ObservatoryID,TimeID,tObs,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tAst,ux,uy,uz
0,1,0,69120000,48000.0,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,-0.009457,-0.005464,0.001583,23.363621,47999.983775,-0.097885,0.990090,0.100704
1,1,0,69127200,48005.0,-0.836785,-0.557668,-0.000054,-1.202187,2.269674,0.290725,-0.009357,-0.005659,0.001558,23.833511,48004.983449,-0.127458,0.986641,0.101460
2,1,0,69134400,48010.0,-0.787422,-0.627462,-0.000056,-1.248709,2.240893,0.298453,-0.009251,-0.005853,0.001533,24.289747,48009.983132,-0.157895,0.982153,0.102200
3,1,0,69141600,48015.0,-0.732357,-0.692671,-0.000054,-1.294694,2.211151,0.306052,-0.009142,-0.006044,0.001507,24.731042,48014.982826,-0.189059,0.976556,0.102932
4,1,0,69148800,48020.0,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,-0.009028,-0.006233,0.001480,25.156619,48019.982530,-0.220819,0.969791,0.103659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48011,16,0,90691200,62980.0,-0.842921,-0.549267,0.000119,-0.813070,2.724700,-0.105745,-0.010217,-0.001563,0.000351,27.244467,62979.981080,0.009167,0.999436,-0.032318
48012,16,0,90698400,62985.0,-0.794209,-0.619483,0.000126,-0.864021,2.716450,-0.103974,-0.010163,-0.001737,0.000357,27.764006,62984.980719,-0.020859,0.999296,-0.031185
48013,16,0,90705600,62990.0,-0.739772,-0.685171,0.000131,-0.914696,2.707332,-0.102170,-0.010107,-0.001910,0.000364,28.265182,62989.980371,-0.051416,0.998224,-0.030103
48014,16,0,90712800,62995.0,-0.680044,-0.745908,0.000132,-0.965081,2.697356,-0.100334,-0.010047,-0.002080,0.000370,28.747137,62994.980037,-0.082410,0.996174,-0.029068


In [6]:
# Column names for testing 
cols_q_obs = ['qObs_x', 'qObs_y', 'qObs_z']
cols_q_ast = ['qAst_x', 'qAst_y', 'qAst_z']
cols_v_ast = ['vAst_x', 'vAst_y', 'vAst_z']
cols_u = ['ux', 'uy', 'uz']

In [7]:
# Extract arrays from Horizons DataFrame
asteroid_id = df_jpl.AsteroidID.values
t_ast = df_jpl.tAst.values
q_obs = df_jpl[cols_q_obs].values
t_obs = df_jpl.tObs.values
u_jpl = df_jpl[cols_u].values
light_time_jpl = df_jpl.LightTime.values

In [8]:
# Build spline for asteroid position according to JPL
spline_q_ast_jpl = make_spline_df(df=df_jpl, cols_spline=cols_q_ast, time_col='tObs', id_col='AsteroidID')

In [9]:
from asteroid_direction import calc_direction, calc_direction_dist, calc_dir_spline, c
from asteroid_direction_test import test_ast_dir

In [10]:
# Calculate asteroid direction using the JPL spline
u_mse, delta_mse = calc_dir_spline(spline_q_ast=spline_q_ast_jpl, q_obs=q_obs, t_obs=t_obs, asteroid_id=asteroid_id, light_time=light_time_jpl, iters=0)
light_time_mse = delta_mse / c

In [11]:
light_time_mse - light_time_jpl

array([-6.46391861e-04, -6.11782659e-05, -5.92848797e-05, ...,
       -7.84527971e-05, -7.53412712e-05, -7.21010628e-05])

In [12]:
# The splined asteroid position using the JPL position spline and JPL light time
q_ast_jpl = spline_q_ast_jpl(t_ast, asteroid_id)

In [14]:
u_mse, delta_mse = calc_direction_dist(q0=q_obs, q1=q_ast_jpl)
light_time_mse = delta_mse / c
# light_time_mse = (t_obs - t_ast)*1440.0

In [15]:
du, dlt = test_ast_dir(name1='JPL', name2='MSE', u1=u_jpl, u2=u_mse, lt1=light_time_jpl, lt2=light_time_mse, verbose=True)


Angle Difference: MSE vs. JPL in arc seconds
*Mean  :     0.855*
 Median:     0.892
 Max   :    13.766
Light time difference in minutes:
 Mean  :  5.19e-05


In [16]:
n0=1
n1=17
mjd0=57000
mjd1=61000
iters=3

In [17]:
df_mse = calc_dir_ast2obs(t_obs=t_obs, asteroid_id=asteroid_id, q_obs=q_obs, iters=iters)

In [18]:
df_mse

,AsteroidID,tObs,ux,uy,uz,LightTime,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z
0,1,48000.0,-0.097935,0.990084,0.100714,23.363004,47999.983776,-0.880028,-0.483807,-0.000050,-1.155143,2.297488,0.282872
1,1,48005.0,-0.127451,0.986642,0.101459,23.833479,48004.983449,-0.836785,-0.557668,-0.000054,-1.202025,2.269773,0.290698
2,1,48010.0,-0.157888,0.982154,0.102199,24.289716,48009.983132,-0.787422,-0.627462,-0.000056,-1.248545,2.240997,0.298426
3,1,48015.0,-0.189052,0.976558,0.102931,24.731012,48014.982826,-0.732357,-0.692671,-0.000054,-1.294530,2.211259,0.306025
4,1,48020.0,-0.220812,0.969792,0.103658,25.156590,48019.982530,-0.672042,-0.752868,-0.000054,-1.339957,2.180571,0.313492
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48011,16,62980.0,0.009171,0.999436,-0.032318,27.244382,62979.981080,-0.842921,-0.549267,0.000119,-0.812878,2.724730,-0.105751
48012,16,62985.0,-0.020854,0.999296,-0.031186,27.763924,62984.980719,-0.794209,-0.619483,0.000126,-0.863825,2.716483,-0.103981
48013,16,62990.0,-0.051412,0.998224,-0.030103,28.265103,62989.980371,-0.739772,-0.685171,0.000131,-0.914498,2.707369,-0.102178
48014,16,62995.0,-0.082405,0.996175,-0.029068,28.747060,62994.980037,-0.680044,-0.745908,0.000132,-0.964881,2.697397,-0.100342


In [19]:
u_mse = df_mse[cols_u].values

In [20]:
du = (u_mse - u_jpl)

In [21]:
err = np.sqrt(np.sum(np.square(du), axis=-1))

In [22]:
idx = np.argmax(err)


In [23]:
# Positions
# q_ast_jpl = df_jpl[cols_q_ast].values
q_ast_mse = df_mse[cols_q_ast].values

q_obs_jpl = df_jpl[cols_q_ast].values
q_obs_mse = df_jpl[cols_q_ast].values

In [24]:
dq_ast = q_ast_mse - q_ast_jpl
dq_obs = q_obs_mse - q_obs_jpl

In [25]:
err_q_obs = np.sqrt(np.sum(np.square(dq_obs), axis=-1))
err_q_ast = np.sqrt(np.sum(np.square(dq_ast), axis=-1))

In [26]:
err_q_obs_mean = np.mean(err_q_obs)
err_q_ast_mean = np.mean(err_q_ast)

In [27]:
print(f'Mean position error:')
print(f'obs: {err_q_obs_mean:5.3e}')
print(f'ast: {err_q_ast_mean:5.3e}')

Mean position error:
obs: 0.000e+00
ast: 1.675e-06


In [29]:
err_q_ast[mask][0:10]

array([9.17092065e-06, 9.25774538e-06, 9.26334125e-06, 9.26844574e-06,
       9.27330392e-06, 9.27782257e-06, 9.28203190e-06, 9.28594228e-06,
       9.28953580e-06, 9.29278044e-06])